In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date

In [45]:
n225 = pd.read_csv('^N225-17-09-2014-29-05-2024.csv', index_col='Date', parse_dates=['Date'])['Adj Close'] # Nikkei 225
hsi = pd.read_csv('^HSI-17-09-2014-29-05-2024.csv', index_col='Date', parse_dates=['Date'])['Adj Close'] # Hang Seng
xwd = pd.read_csv('XWD.TO-17-09-2014-29-05-2024.csv', index_col='Date', parse_dates=['Date'])['Adj Close'] # MSCI World
eem = pd.read_csv('EEM-17-09-2014-29-05-2024.csv', index_col='Date', parse_dates=['Date'])['Adj Close'] # Emerging Markets
sp500 = pd.read_csv('^GSPC-17-09-2014-29-05-2024.csv', index_col='Date', parse_dates=['Date'])['Adj Close'] # S&P 500
dji = pd.read_csv('^DJI-17-09-2014-29-05-2024.csv', index_col='Date', parse_dates=['Date'])['Adj Close'] # Dow Jones
rui = pd.read_csv('^RUI-17-09-2014-29-05-2024.csv', index_col='Date', parse_dates=['Date'])['Adj Close'] # Russell 1000
stoxx600 = pd.read_csv('^STOXX-17-09-2014-29-05-2024.csv', index_col='Date', parse_dates=['Date'])['Adj Close'] # Stoxx 600
stoxx50 = pd.read_csv('^STOXX50E-17-09-2014-29-05-2024.csv', index_col='Date', parse_dates=['Date'])['Adj Close'] # Stoxx 50
ndx = pd.read_csv('^NDX-17-09-2014-29-05-2024.csv', index_col='Date', parse_dates=['Date'])['Adj Close'] # Nasdaq 100

n225.name = 'Nikkei 225'
hsi.name = 'Hang Seng'
xwd.name = 'MSCI World'
eem.name = 'Emerging Markets'
sp500.name = 'S&P 500'
stoxx600.name = 'Stoxx 600'
ndx.name = 'Nasdaq 100'

In [181]:
import pandas as pd
import numpy as np
from itertools import combinations

prices = pd.concat([n225, hsi, xwd, sp500, stoxx600, ndx], axis=1).ffill().dropna()
prices = prices[sorted(prices.columns)]
ret = (prices / prices.shift() - 1)
window = 20
rolling_corr = ret.rolling(window).corr()

pairs = list(combinations(rolling_corr.columns, 2))

# Initialize a dictionary to store the data
data_dict = {
    (date, f'{pair[0]}-{pair[1]}'): rolling_corr.loc[(date, pair[0]), pair[1]]
    for date in rolling_corr.index.levels[0] for pair in pairs
}

# Convert the dictionary to a DataFrame
index = pd.MultiIndex.from_tuples(data_dict.keys(), names=['Date', 'Pair'])
values = list(data_dict.values())
result_df = pd.DataFrame(values, index=index, columns=['Correlation'])

pair_ts = result_df.xs('Nikkei 225-S&P 500', level='Pair')
result_df


Correlation
Date       Pair                             
2014-09-17 Hang Seng-MSCI World          NaN
           Hang Seng-Nasdaq 100          NaN
           Hang Seng-Nikkei 225          NaN
           Hang Seng-S&P 500             NaN
           Hang Seng-Stoxx 600           NaN
...                                      ...
2024-05-29 Nasdaq 100-S&P 500       0.932495
           Nasdaq 100-Stoxx 600     0.304625
           Nikkei 225-S&P 500      -0.131303
           Nikkei 225-Stoxx 600     0.391900
           S&P 500-Stoxx 600        0.400847

[37845 rows x 1 columns]

In [183]:
# correls for the pair Nikkei 225-S&P 500
pair_ts

,Correlation
Date,
2014-09-17,NaN
2014-09-18,NaN
2014-09-19,NaN
2014-09-22,NaN
2014-09-23,NaN
...,...
2024-05-23,-0.244551
2024-05-24,-0.349522
2024-05-27,-0.353768


In [179]:
a = pd.DataFrame({
    'a': [1, 2, 1.2],
    'b': [1, 2, 3],
})
display(a)
a.rolling(3).corr()

,a,b
0,1.0,1
1,2.0,2
2,1.2,3


a         b
0 a       NaN       NaN
  b       NaN       NaN
1 a       NaN       NaN
  b       NaN       NaN
2 a  1.000000  0.188982
  b  0.188982  1.000000